In [7]:
    if __name__ == '__main__':
        %reload_ext importable
    from contextlib import contextmanager
    from functools import partial
    from importable import finder, unload
    from IPython import display, get_ipython
    from IPython.core.interactiveshell import InteractiveShell
    from mistune import Markdown, Renderer
    from nbformat.v4 import new_notebook, new_code_cell
    from nbconvert import export, get_exporter
    from nbconvert.exporters.python import PythonExporter    
    from toolz.curried import compose, do
    from types import MethodType

In [8]:
    def run_cell(ip, enter=[], exit=[]):
        runner = run(enter, exit)
        def _run_cell(self, text, store_history=False, silent=False, shell_features=True):
            with runner(text) as code:
                    return InteractiveShell.run_cell(self, code, store_history, silent, shell_features)
        ip.run_cell = MethodType(_run_cell, ip)

    def run(enter=[], exit=[]):
        @contextmanager
        def _run(code):
            for fn in enter:
                _, code = code, fn(code)
            yield code
            for fn in exit:
                fn()
        return _run

In [9]:
    def indents(code):
        """Determine the indent length of the last line."""
        if code:
            str = list(filter(lambda s: s.strip(), code.splitlines()))
            if str:  
                return len(str[-1])-len(str[-1].lstrip())
        return 0

    def rstrip(text):
        """Remove whitespace for diffing later."""
        return '\n'.join(map(str.rstrip, map(str, text.splitlines())))

In [10]:
    class ExtractCode(Renderer):
        """A mistune.Renderer to accumulate lines of code in a Markdown document."""
        code = """"""
        def block_code(self, code, lang=None):
            self.code += code + '\n'
            return super(ExtractCode, self).block_code(code, lang)

        def codespan(self, code):
            self.code += ' '*indents(self.code) + code + '\n'
            return super(ExtractCode, self).codespan(code)

    class CodeOnly(Markdown):
        """A mistune.Markdown processor for literate programming."""
        def render(self, text, **kwargs):
            """Render ingests Markdown tests and returns executable portions"""
            text=rstrip(text)
            setattr(self.renderer, 'code', """""") or super(CodeOnly, self).render(text, **kwargs)
            data =  '\n'.join(str.rstrip('    '+s) for s in self.renderer.code.splitlines())
            return PythonExporter().from_notebook_node(
                new_notebook(cells=[new_code_cell(data)])
            )[0]

In [11]:
    literate = CodeOnly(renderer=ExtractCode(), escape=False)

In [12]:
    @do
    def disp(code, key='data'):
        from pathlib import Path
        try:
            if Path(code).exists(): key = 'filename'
        except OSError:
            pass
        display.display(display.Markdown(**{key: code}))

In [13]:
    exporter = get_exporter('script')

    def nb_to_python(callable, nb):
        new = []
        for cell in nb.cells:
            if cell['cell_type'] == 'code':
                cell.source = callable(
                    '\n'.join(cell.source) if isinstance(cell.source, list) else cell.source)
                new.append(cell)
            nb.cells = new
        return export(exporter, nb)[0].encode('utf-8')

    def loader(render):
        def wrapped(self, path):
            nonlocal render
            with open(self.path) as f:
                out = nb_to_python(render, __import__('nbformat').read(f, 4))
            return out
        return unload(['ipynb']) or finder('ipynb')(wrapped)

In [15]:
    class Execute(__import__('nbconvert').preprocessors.execute.ExecutePreprocessor):
        def run_cell(self, code, index=0):
            code.source = literate.render(code.source)
            return super().run_cell(code, index)

In [16]:
    def load_ipython_extension(ip=get_ipython()):
        run_cell(ip, [disp, literate.render])
        loader(literate.render)